# Evaluation pipeline

## 0 Imports and Constants

## 1 Load Baseline Model and Data

## 2 Train and Test Baseline Model

### 2.1 Train on Real, Test on Synthetic

### 2.2 Train on Synthetic, Test on Real

## 3 Visualize Synthetic vs Real Data

## 4 Save Evaluation Data